Filter BLASTp results to identify Reciprocal Best Hits (RBHs)

In [ ]:
import pandas as pd
import os

os.getcwd()

In [ ]:
# Function to identify RBH between two blastp output files
def find_reciprocal_hits(file1, file2, output_rbh, output_rbh_no, species_a, species_b):
    
    #file1 & file2 = Paths to input files containing BLAST results: a2b.txt & b2a.txt
    #output_rbh = Path to output file to save RBHs
    #output_rbh_no = Path to output file to save genes/peptides from species_a with NO RBHs
    #species_a & species_b = Strings containing name "genus_species"
    
    # Load BLASTP results into DataFrames
    col_names = ["query", "subject", "perc_identity", "alignment_length", 
                 "mismatches", "gap_opens", "q_start", "q_end", 
                 "s_start", "s_end", "evalue", "bit_score"]
    
       
    df1 = pd.read_csv(file1, sep="\t", names=col_names)
    df2 = pd.read_csv(file2, sep="\t", names=col_names)
    
    # Extract the best hits from both files by keeping only best hit per query based on e-value
    # Ties are broken by selecting the highest bit score (% similarity)
    best_hits1 = df1.sort_values(by = ["query", "evalue", "bit_score"],
                                 ascending=[True, True, False]).drop_duplicates(subset=["query"], keep="first")
    best_hits2 = df2.sort_values(by = ["query", "evalue", "bit_score"],
                                 ascending=[True, True, False]).drop_duplicates(subset=["query"], keep="first")
    
    # Create dictionaries for quick lookup
    best_hits1_dict = dict(zip(best_hits1["query"], best_hits1["subject"]))
    best_hits2_dict = dict(zip(best_hits2["query"], best_hits2["subject"]))
    
    # Find reciprocal best hits
    rbh = []
    rbh_no = [] # to sore queries without a RBH
    for query, subject in best_hits1_dict.items():
        if best_hits2_dict.get(subject) == query:
            rbh.append((query, subject))
        else:
            rbh_no.append(query)
    
    # Save the results with RBH
    rbh_df = pd.DataFrame(rbh, columns=[f"{species_a}", f"{species_b}"])
    
  
    rbh_df.to_csv(output_rbh, sep="\t", index=False)
    print(f"Reciprocal best hits: {output_rbh}")
    
    # Save the queries with no reciprocal match
    rbh_no_df = pd.DataFrame(rbh_no, columns=[f"{species_a}"])
    rbh_no_df.to_csv(output_rbh_no, sep="\t", index=False)
    print(f"Queries from {species_a} with no RBH: {output_rbh_no}")
    
    print(rbh_df.head())
    print(rbh_no_df.head())

In [ ]:
# File paths
base_path = "/cluster/tufts/dopmanlab/Jacob/onub_ortholog_id/output/"

file1 = base_path + "Onub2Bmor_blastp_v2.txt"
file2 = base_path + "Bmor2Onub_blastp_v2.txt"
output_rbh = base_path + "OnubBmor_rbh_v2.txt"
output_rbh_no = base_path + "OnubBmor_rbh_no_v2.txt"
species_a = "ostrinia_nubilalis"
species_b = "bombyx_mori"

In [ ]:
find_reciprocal_hits(file1, file2, output_rbh, output_rbh_no, species_a, species_b)